In [1]:
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 36.2 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/11.1 MB ? eta -:--:--

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 164.9 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 23.9 MB/s eta 0:00:00


   ━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/34.5 MB 147.6 MB/s eta 0:00:01

   ━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━ 14.4/34.5 MB 135.0 MB/s eta 0:00:01

   ━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━ 20.0/34.5 MB 162.8 MB/s eta 0:00:01

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━ 30.4/34.5 MB 151.8 MB/s eta 0:00:01

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 149.2 MB/s eta 0:00:00


In [2]:
from operator import itemgetter
import pickle
import numpy as np

In [3]:
# Carregar variáveis do arquivo pickle
with open('treinados.pkl', 'rb') as f:
    data = pickle.load(f)

rf = data['rf']

In [4]:
# Carregar variáveis do arquivo pickle
with open('model.pkl', 'rb') as f:
    data = pickle.load(f)

model_db = data['model_db']

In [5]:
# Carregar variáveis do arquivo pickle
with open('stats.pkl', 'rb') as f:
    data = pickle.load(f)

team_stats_raw = data['team_stats_raw']

In [6]:
table = {'A': [['Argentina', 0, []],
  ['Peru', 0, []],
  ['Chile', 0, []],
  ['Canada', 0, []]],
 'B': [['Mexico', 0, []],
  ['Ecuador', 0, []],
  ['Venezuela', 0, []],
  ['Jamaica', 0, []]],
 'C': [['United States', 0, []],
  ['Uruguay', 0, []],
  ['Panama', 0, []],
  ['Bolivia', 0, []]],
 'D': [['Brazil', 0, []],
  ['Colombia', 0, []],
  ['Paraguay', 0, []],
  ['Costa Rica', 0, []]]}

matches = [('A', 'Argentina', 'Canada'),
 ('A', 'Peru', 'Chile'),
 ('A', 'Peru', 'Canada'),
 ('A', 'Chile', 'Argentina'),
 ('A', 'Argentina', 'Peru'),
 ('A', 'Canada', 'Chile'),
 
 ('B', 'Ecuador', 'Venezuela'),
 ('B', 'Mexico', 'Jamaica'),
 ('B', 'Ecuador', 'Jamaica'),
 ('B', 'Venezuela', 'Mexico'),
 ('B', 'Mexico', 'Ecuador'),
 ('B', 'Jamaica', 'Venezuela'),
 
 ('C', 'United States', 'Bolivia'),
 ('C', 'Uruguay', 'Panama'),
 ('C', 'Panama', 'United States'),
 ('C', 'Uruguay', 'Bolivia'),
 ('C', 'United States', 'Uruguay'),
 ('C', 'Bolivia', 'Panama'),
 
 ('D', 'Colombia', 'Paraguay'),
 ('D', 'Brazil', 'Costa Rica'),
 ('D', 'Colombia', 'Costa Rica'),
 ('D', 'Paraguay', 'Brazil'),
 ('D', 'Brazil', 'Colombia'),
 ('D', 'Costa Rica', 'Paraguay')]

In [7]:
def find_stats(team_1):
    past_games = team_stats_raw[(team_stats_raw["team"] == team_1)].sort_values("date")
    last5 = team_stats_raw[(team_stats_raw["team"] == team_1)].sort_values("date").tail(5)

    team_1_rank = past_games["rank"].values[-1]
    team_1_goals = past_games.score.mean()
    team_1_goals_l5 = last5.score.mean()
    team_1_goals_suf = past_games.suf_score.mean()
    team_1_goals_suf_l5 = last5.suf_score.mean()
    team_1_rank_suf = past_games.rank_suf.mean()
    team_1_rank_suf_l5 = last5.rank_suf.mean()
    team_1_gp_rank = past_games.points_by_rank.mean()
    team_1_gp_rank_l5 = last5.points_by_rank.mean()

    return [team_1_rank, team_1_goals, team_1_goals_l5, team_1_goals_suf, team_1_goals_suf_l5, team_1_rank_suf, team_1_rank_suf_l5, team_1_gp_rank, team_1_gp_rank_l5]

In [8]:
def find_features(team_1, team_2):
    rank_dif = team_1[0] - team_2[0]
    goals_dif = team_1[1] - team_2[1]
    goals_dif_l5 = team_1[2] - team_2[2]
    goals_suf_dif = team_1[3] - team_2[3]
    goals_suf_dif_l5 = team_1[4] - team_2[4]
    goals_per_ranking_dif = (team_1[1]/team_1[5]) - (team_2[1]/team_2[5])
    dif_rank_agst = team_1[5] - team_2[5]
    dif_rank_agst_l5 = team_1[6] - team_2[6]
    dif_gp_rank = team_1[7] - team_2[7]
    dif_gp_rank_l5 = team_1[8] - team_2[8]
    
    return [rank_dif, goals_dif, goals_dif_l5, goals_suf_dif, goals_suf_dif_l5, goals_per_ranking_dif, dif_rank_agst, dif_rank_agst_l5, dif_gp_rank, dif_gp_rank_l5, 1, 0]

In [9]:
advanced_group = []
last_group = ""

for k in table.keys():
    for t in table[k]:
        t[1] = 0
        t[2] = []
        
for teams in matches:
    draw = False
    team_1 = find_stats(teams[1])
    team_2 = find_stats(teams[2])

    features_g1 = find_features(team_1, team_2)
    features_g2 = find_features(team_2, team_1)

    probs_g1 = rf.predict_proba([features_g1])
    probs_g2 = rf.predict_proba([features_g2])
    
    team_1_prob_g1 = probs_g1[0][0]
    team_1_prob_g2 = probs_g2[0][1]
    team_2_prob_g1 = probs_g1[0][1]
    team_2_prob_g2 = probs_g2[0][0]

    team_1_prob = (probs_g1[0][0] + probs_g2[0][1])/2
    team_2_prob = (probs_g2[0][0] + probs_g1[0][1])/2
    
    if ((team_1_prob_g1 > team_2_prob_g1) & (team_2_prob_g2 > team_1_prob_g2)) | ((team_1_prob_g1 < team_2_prob_g1) & (team_2_prob_g2 < team_1_prob_g2)):
        draw=True
        for i in table[teams[0]]:
            if i[0] == teams[1] or i[0] == teams[2]:
                i[1] += 1
                
    elif team_1_prob > team_2_prob:
        winner = teams[1]
        winner_proba = team_1_prob
        for i in table[teams[0]]:
            if i[0] == teams[1]:
                i[1] += 3
                
    elif team_2_prob > team_1_prob:  
        winner = teams[2]
        winner_proba = team_2_prob
        for i in table[teams[0]]:
            if i[0] == teams[2]:
                i[1] += 3
    
    for i in table[teams[0]]: #adding criterio de desempate (probs por jogo)
            if i[0] == teams[1]:
                i[2].append(team_1_prob)
            if i[0] == teams[2]:
                i[2].append(team_2_prob)

    if last_group != teams[0]:
        if last_group != "":
            print("\n")
            print("Do grupo %s avançam: "%(last_group))
            
            for i in table[last_group]: #adding crieterio de desempate
                i[2] = np.mean(i[2])
            
            final_points = table[last_group]
            final_table = sorted(final_points, key=itemgetter(1, 2), reverse = True)
            advanced_group.append([final_table[0][0], final_table[1][0]])
            for i in final_table:
                print("%s -------- %d"%(i[0], i[1]))
        print("\n")
        print("-"*10+" Simulando o Grupo %s "%(teams[0])+"-"*10)
        
        
    if draw == False:
        print("Grupo %s - %s vs. %s: Vencedor %s com %.2f de probabilidade"%(teams[0], teams[1], teams[2], winner, winner_proba))
    else:
        print("Grupo %s - %s vs. %s: Empate"%(teams[0], teams[1], teams[2]))
    last_group =  teams[0]

print("\n")
print("Do grupo %s avançam: "%(last_group))

for i in table[last_group]: #adding crieterio de desempate
    i[2] = np.mean(i[2])
            
final_points = table[last_group]
final_table = sorted(final_points, key=itemgetter(1, 2), reverse = True)
advanced_group.append([final_table[0][0], final_table[1][0]])
for i in final_table:
    print("%s -------- %d"%(i[0], i[1]))



---------- Simulando o Grupo A ----------
Grupo A - Argentina vs. Canada: Vencedor Argentina com 0.81 de probabilidade
Grupo A - Peru vs. Chile: Vencedor Peru com 0.57 de probabilidade
Grupo A - Peru vs. Canada: Empate
Grupo A - Chile vs. Argentina: Vencedor Argentina com 0.82 de probabilidade


Grupo A - Argentina vs. Peru: Vencedor Argentina com 0.75 de probabilidade
Grupo A - Canada vs. Chile: Empate


Do grupo A avançam: 
Argentina -------- 9
Peru -------- 4
Canada -------- 2
Chile -------- 1


---------- Simulando o Grupo B ----------
Grupo B - Ecuador vs. Venezuela: Vencedor Ecuador com 0.65 de probabilidade
Grupo B - Mexico vs. Jamaica: Vencedor Mexico com 0.72 de probabilidade
Grupo B - Ecuador vs. Jamaica: Vencedor Ecuador com 0.73 de probabilidade


Grupo B - Venezuela vs. Mexico: Vencedor Mexico com 0.64 de probabilidade
Grupo B - Mexico vs. Ecuador: Empate
Grupo B - Jamaica vs. Venezuela: Vencedor Venezuela com 0.57 de probabilidade


Do grupo B avançam: 
Ecuador -------- 7
Mexico -------- 7
Venezuela -------- 3
Jamaica -------- 0


---------- Simulando o Grupo C ----------
Grupo C - United States vs. Bolivia: Vencedor United States com 0.82 de probabilidade
Grupo C - Uruguay vs. Panama: Vencedor Uruguay com 0.64 de probabilidade
Grupo C - Panama vs. United States: Vencedor United States com 0.63 de probabilidade
Grupo C - Uruguay vs. Bolivia: Vencedor Uruguay com 0.73 de probabilidade


Grupo C - United States vs. Uruguay: Empate
Grupo C - Bolivia vs. Panama: Vencedor Panama com 0.71 de probabilidade


Do grupo C avançam: 
United States -------- 7
Uruguay -------- 7
Panama -------- 3
Bolivia -------- 0


---------- Simulando o Grupo D ----------
Grupo D - Colombia vs. Paraguay: Vencedor Colombia com 0.65 de probabilidade
Grupo D - Brazil vs. Costa Rica: Vencedor Brazil com 0.79 de probabilidade
Grupo D - Colombia vs. Costa Rica: Vencedor Colombia com 0.75 de probabilidade


Grupo D - Paraguay vs. Brazil: Vencedor Brazil com 0.75 de probabilidade
Grupo D - Brazil vs. Colombia: Empate
Grupo D - Costa Rica vs. Paraguay: Empate


Do grupo D avançam: 
Brazil -------- 7
Colombia -------- 7
Paraguay -------- 1
Costa Rica -------- 1


In [10]:
advanced = advanced_group

In [11]:
playoffs = {"s Quartas de Final": [], " Semi-Final": [], " Final": []}

for p in playoffs.keys():
    playoffs[p] = []

actual_round = ""
next_rounds = []

for p in playoffs.keys():
    if p == "s Quartas de Final":
        control = []
        for a in range(0, len(advanced*2), 1):
            if a < len(advanced):
                if a % 2 == 0:
                    control.append((advanced*2)[a][0])
                else:
                    control.append((advanced*2)[a][1])
            else:
                if a % 2 == 0:
                    control.append((advanced*2)[a][1])
                else:
                    control.append((advanced*2)[a][0])

        playoffs[p] = [[control[c], control[c+1]] for c in range(0, len(control)-1, 1) if c%2 == 0]
        
        for i in range(0, len(playoffs[p]), 1):
            game = playoffs[p][i]
            
            home = game[0]
            away = game[1]
            team_1 = find_stats(home)
            team_2 = find_stats(away)

            features_g1 = find_features(team_1, team_2)
            features_g2 = find_features(team_2, team_1)
            
            probs_g1 = rf.predict_proba([features_g1])
            probs_g2 = rf.predict_proba([features_g2])
            
            team_1_prob = (probs_g1[0][0] + probs_g2[0][1])/2
            team_2_prob = (probs_g2[0][0] + probs_g1[0][1])/2
            
            if actual_round != p:
                print("-"*10)
                print("Simulação da%s"%(p))
                print("-"*10)
                print("\n")
            
            if team_1_prob < team_2_prob:
                print("%s vs. %s: %s avança com a probabilidade de %.2f"%(home, away, away, team_2_prob))
                next_rounds.append(away)
            else:
                print("%s vs. %s: %s avança com a probabilidade de %.2f"%(home, away, home, team_1_prob))
                next_rounds.append(home)
            
            game.append([team_1_prob, team_2_prob])
            playoffs[p][i] = game
            actual_round = p
        
    else:
        playoffs[p] = [[next_rounds[c], next_rounds[c+1]] for c in range(0, len(next_rounds)-1, 1) if c%2 == 0]
        next_rounds = []
        for i in range(0, len(playoffs[p])):
            game = playoffs[p][i]
            home = game[0]
            away = game[1]
            team_1 = find_stats(home)
            team_2 = find_stats(away)
            
            features_g1 = find_features(team_1, team_2)
            features_g2 = find_features(team_2, team_1)
            
            probs_g1 = rf.predict_proba([features_g1])
            probs_g2 = rf.predict_proba([features_g2])
            
            team_1_prob = (probs_g1[0][0] + probs_g2[0][1])/2
            team_2_prob = (probs_g2[0][0] + probs_g1[0][1])/2
            
            if actual_round != p:
                print("-"*10)
                print("Simulação da%s"%(p))
                print("-"*10)
                print("\n")
            
            if team_1_prob < team_2_prob:
                print("%s vs. %s: %s avança com a probabilidade de %.2f"%(home, away, away, team_2_prob))
                next_rounds.append(away)
            else:
                print("%s vs. %s: %s avança com a probabilidade de %.2f"%(home, away, home, team_1_prob))
                next_rounds.append(home)
            game.append([team_1_prob, team_2_prob])
            playoffs[p][i] = game
            actual_round = p
            

----------
Simulação das Quartas de Final
----------


Argentina vs. Mexico: Argentina avança com a probabilidade de 0.62
United States vs. Colombia: United States avança com a probabilidade de 0.53


Peru vs. Ecuador: Ecuador avança com a probabilidade de 0.57
Uruguay vs. Brazil: Brazil avança com a probabilidade de 0.58
----------
Simulação da Semi-Final
----------


Argentina vs. United States: Argentina avança com a probabilidade de 0.54
Ecuador vs. Brazil: Brazil avança com a probabilidade de 0.65
----------
Simulação da Final
----------


Argentina vs. Brazil: Argentina avança com a probabilidade de 0.51
